In [ ]:
import json
from scipy import sparse
import numpy as np
from gensim.test.utils import datapath
from gensim import utils
import gensim.models
from tqdm import tqdm
from node2vec import Node2Vec
import networkx as nx
from itertools import cycle

In [ ]:
#!pip install gensim --user

In [ ]:
config = json.load(open('../config/data-params.json'))

config['master_matrix'] =  '.' + config['master_matrix']


In [ ]:
%%time

train_A = sparse.load_npz(config['master_matrix'] + '/train_A.npz').tocsr()
train_B = sparse.load_npz(config['master_matrix'] + '/train_B.npz').tocsc()
train_P = sparse.load_npz(config['master_matrix'] + '/train_P.npz').tocsc()
train_A_csc = train_A.tocsc()

test_A = sparse.load_npz(config['master_matrix'] + '/test_A.npz').tocsr()

In [ ]:
%%time
train_APA_T = train_A.dot(train_P).dot(train_A.T)
test_APA_T = test_A.dot(train_P).dot(train_A.T)

In [ ]:
# ABPBA
def generate_corpus(walk_length=5):
    while True:
              
        app = np.random.choice(np.arange(train_A.shape[0]))
        
        path = f'app_{app}'
        
        for i in range(walk_length):
        
            api_i = np.random.choice(np.nonzero(train_A[app])[1])
            api_bi = np.random.choice(np.nonzero(train_B[:, api_i])[0])
            api_p = np.random.choice(np.nonzero(train_P[:, api_bi])[0])
            api_bj = np.random.choice(np.nonzero(train_B[:, api_p])[0])
            app = np.random.choice(np.nonzero(train_A_csc[:, api_bj])[0])

            path += f' api_{api_i} api_{api_bi} api_{api_p} api_{api_bj} app_{app}'
            
        yield path

In [ ]:
# ABPBA
def generate_corpus_limit_length():
    while True:
        app_i = np.random.choice(np.arange(train_A.shape[0]))
        api_i = np.random.choice(np.nonzero(train_A[app_i])[1])
        api_bi = np.random.choice(np.nonzero(train_B[:, api_i])[0])
        api_p = np.random.choice(np.nonzero(train_P[:, api_bi])[0])
        api_bj = np.random.choice(np.nonzero(train_B[:, api_p])[0])
        app_j = np.random.choice(np.nonzero(train_A_csc[:, api_bj])[0])

        yield f'app_{app_i} api_{api_i} api_{api_bi} api_{api_p} api_{api_bj} app_{app_j}'
        

In [ ]:
corpus_function = generate_corpus()

In [ ]:
next(corpus_function)

In [ ]:
%%time
f = open('test.cor', 'w')
for _ in tqdm(range(10000)):
    f.write(next(corpus_function) + '\n')
f.close()

In [ ]:
!head test.cor

In [ ]:
class MyCorpus(object):
    """An interator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = './test.cor'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield line.strip().split(' ')

In [ ]:
%%time
sentences = MyCorpus()
model = gensim.models.Word2Vec(sentences=sentences, size=4)

In [ ]:
next(sentences.__iter__())

In [ ]:
model.wv.similarity('app_500', 'app_100')

In [ ]:
print(model.wv.most_similar(positive=['app_334'], topn=10))

In [ ]:
model_with_loss = gensim.models.Word2Vec(
    sentences,
    min_count=1,
    compute_loss=True,
    hs=0,
    sg=1,
    seed=42
)

# getting the training loss value
training_loss = model_with_loss.get_latest_training_loss()
print(training_loss)